In [2]:
import torch
import random
import numpy as np
import os
import torch.nn as nn
from glob import glob
from tqdm import tqdm
from os.path import join
import os.path as osp
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from util import *

def calculate_metric(y_label, y_pred_label):
    mic_accuracy = accuracy_score(y_label, y_pred_label)
    macro_precision = precision_score(y_label, y_pred_label, average='macro')
    macro_recall = recall_score(y_label, y_pred_label, average='macro')
    macro_f1 = f1_score(y_label, y_pred_label, average='macro')
    return mic_accuracy, macro_precision, macro_recall, macro_f1

def eval_spectrum(data, model_file):
    pkl_file = open(data, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    X = data["data"]
    y = data["label"]
    with open(model_file, 'rb') as f:
        svclassifier_r = pickle.load(f)
    # print("Loaded.")
    SVM_score = svclassifier_r.score(X, y)
    # print("accuracy by score function: " + str(SVM_score))
    ##### CALCULATE METRIC:
    y_pred = svclassifier_r.predict(X)
    acc, pre, rec, f1 = calculate_metric(y, y_pred)
    # print("accuracy: " + str(acc))
    # print("precision: " + str(pre))
    # print("recall: " + str(rec))
    # print("f1 score: " + str(f1))
    return SVM_score, acc, pre, rec, f1

def extract_test_features(test_dir: str, output_test: str, brightness: float, contrast: float, std=0, missing_value=0, quality=100, type_transform='noise'):
    testset = get_test_path(test_dir=test_dir)
    real_path, fake_path = split_real_fake(train_paths=testset)
    extract_spectrum_features(real_path, fake_path, -1, output_test, brightness=brightness, contrast=contrast, std=std, miss_size=missing_value, quality=quality, type_transform=type_transform)
    return output_test


test_dir_template = "/mnt/disk1/doan/phucnp/Dataset/my_extend_data/extend_data_train/{}/test"

checkpoints = {
    "deepfake": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/deepfake/spectrum/c_2.000000",
    "faceswap_2d": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/faceswap_2d/spectrum/c_2.000000",
    "3dmm": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/3dmm/spectrum/c_2.000000",
    "faceswap_3d": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/faceswap_3d/spectrum/c_2.000000",
    "monkey": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/monkey/spectrum/c_2.000000",
    "reenact": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/reenact/spectrum/c_2.000000",
    "stargan": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/stargan/spectrum/c_2.000000",
    "x2face": "/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/checkpoint/x2face/spectrum/c_2.000000"
}

brightness = 1.0
contrast = 1.0
std= 0
missing_value = 0.0
qualitys=[100, 90, 80, 70, 60, 50]
type_transform = 'compression'

for quality in qualitys:
    print("\n*********************************************************")
    print("             quality: ", quality)
    for technique in ['deepfake', '3dmm', 'faceswap_2d', 'faceswap_3d', 'monkey', 'reenact', 'stargan', 'x2face']:
        checkpoint = checkpoints[technique]
        print("\n\n ==================================== {} CHECKPOINT =========================".format(technique))

        my_break = False
        # filename = '/contrast{}.pkl'.format(contrast)
        # filename = '/noise{}.pkl'.format(std)
        # filename = '/missing{}.pkl'.format(missing_value)
        filename = '/compression{}.pkl'.format(quality)
        output_test = '/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/ml_technique/spectrum/output_features/' + get_datasetname(test_dir_template.format(technique)) + filename
        test_features = extract_test_features(test_dir=test_dir_template.format(technique), output_test=output_test, brightness = brightness, contrast=contrast, std=std, missing_value=missing_value, quality=quality, type_transform=type_transform)        
        
        for pkl_file in sorted(glob(join(checkpoint, '*/*.pkl')), key=lambda pkl: int(pkl.split('/')[-2][-1])):
            if 'model' not in pkl_file:
                continue
                
            fold_id = pkl_file.split('/')[-2][-1]
            ####
            SVM_score, acc, pre, rec, f1 = eval_spectrum(test_features, pkl_file)
            print("            pt file: ", '/'.join(pkl_file.split('/')[-2:]))
            print("    FOLD: {}    -   ".format(fold_id))
            print("            accuracy =   {:.6f}       |  {:6f}".format(SVM_score, acc))
            print("            precision =  {:.6f}  |   recall =    {:.6f}   |  f1 =    {:.6f}".format(pre, rec, f1))

            # if brightness == 1.0:
            #     from_acc = float(pkl_file.split('/')[-2].split('_')[0])
            #     # if abs(round(loss, 4) - round(bestloss, 4)) > 0.001:
            #     #     print("         Error in loss: {}, {}".format(loss, bestloss))
            #     #     my_break = True
            #     if abs(round(from_acc, 4) - round(acc, 4)) > 0.001:
            #         print("         Error in acc: {}, {}".format(from_acc, acc))
            #         my_break = True

            #     if my_break:
            #         break



        if my_break:
            break


*********************************************************
             quality:  100


 ==================================== deepfake CHECKPOINT =========================


100%|██████████| 5000/5000 [00:24<00:00, 202.72it/s]


            pt file:  0.9876_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.987500       |  0.987500
            precision =  0.987626  |   recall =    0.987489   |  f1 =    0.987499
            pt file:  0.9854_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.985900       |  0.985900
            precision =  0.986066  |   recall =    0.985888   |  f1 =    0.985899
            pt file:  0.9858_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.985800       |  0.985800
            precision =  0.985948  |   recall =    0.985788   |  f1 =    0.985799
            pt file:  0.988_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.988100       |  0.988100
            precision =  0.988214  |   recall =    0.988090   |  f1 =    0.988099
            pt file:  0.987_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.987800       |  0.987800
            precision =  0.987905  |   recall =    0.987790   |  f1 =    0.987799




100%|██████████| 5000/5000 [00:23<00:00, 216.01it/s]


            pt file:  0.8929_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.893900       |  0.893900
            precision =  0.893902  |   recall =    0.893900   |  f1 =    0.893900
            pt file:  0.8934_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.893400       |  0.893400
            precision =  0.893403  |   recall =    0.893400   |  f1 =    0.893400
            pt file:  0.892_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.892600       |  0.892600
            precision =  0.892606  |   recall =    0.892600   |  f1 =    0.892600
            pt file:  0.8928_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.893500       |  0.893500
            precision =  0.893500  |   recall =    0.893500   |  f1 =    0.893500
            pt file:  0.8935_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.894000       |  0.894000
            precision =  0.894011  |   recall =    0.894000   |  f1 =    0.893999



...

            pt file:  0.9513_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.808500       |  0.808500
            precision =  0.860594  |   recall =    0.791382   |  f1 =    0.793930
            pt file:  0.9511_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.806800       |  0.806800
            precision =  0.859095  |   recall =    0.789591   |  f1 =    0.791994
            pt file:  0.9504_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.804800       |  0.804800
            precision =  0.859259  |   recall =    0.787243   |  f1 =    0.789377
            pt file:  0.9516_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.808300       |  0.808300
            precision =  0.860760  |   recall =    0.791128   |  f1 =    0.793644
            pt file:  0.9517_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.812100       |  0.812100
            precision =  0.862870  |   recall =    0.795290   |  f1 =    0.798172


100%|██████████| 5000/5000 [00:46<00:00, 107.18it/s]


            pt file:  0.9092_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.696800       |  0.696800
            precision =  0.794919  |   recall =    0.696800   |  f1 =    0.669294
            pt file:  0.9106_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.695400       |  0.695400
            precision =  0.794044  |   recall =    0.695400   |  f1 =    0.667515
            pt file:  0.9088_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.701500       |  0.701500
            precision =  0.796183  |   recall =    0.701500   |  f1 =    0.675572
            pt file:  0.9118_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.696500       |  0.696500
            precision =  0.794368  |   recall =    0.696500   |  f1 =    0.668987
            pt file:  0.9097_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.697100       |  0.697100
            precision =  0.794252  |   recall =    0.697100   |  f1 =    0.669849


100%|██████████| 5000/5000 [00:26<00:00, 185.45it/s]


            pt file:  0.923_fold_0/model.pkl
    FOLD: 0    -   
            accuracy =   0.745100       |  0.745100
            precision =  0.826539  |   recall =    0.745100   |  f1 =    0.728150
            pt file:  0.9229_fold_1/model.pkl
    FOLD: 1    -   
            accuracy =   0.742900       |  0.742900
            precision =  0.823780  |   recall =    0.742900   |  f1 =    0.725775
            pt file:  0.9232_fold_2/model.pkl
    FOLD: 2    -   
            accuracy =   0.746100       |  0.746100
            precision =  0.827176  |   recall =    0.746100   |  f1 =    0.729332
            pt file:  0.9231_fold_3/model.pkl
    FOLD: 3    -   
            accuracy =   0.746900       |  0.746900
            precision =  0.825669  |   recall =    0.746900   |  f1 =    0.730611
            pt file:  0.9226_fold_4/model.pkl
    FOLD: 4    -   
            accuracy =   0.743300       |  0.743300
            precision =  0.823968  |   recall =    0.743300   |  f1 =    0.726260
